# Скачиваем данные

In [ ]:
!wget https://se.ifmo.ru/~s367390/resumes.zip

--2023-11-19 03:55:14--  https://se.ifmo.ru/~s367390/resumes.zip
Resolving se.ifmo.ru (se.ifmo.ru)... 77.234.196.4
Connecting to se.ifmo.ru (se.ifmo.ru)|77.234.196.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 94318691 (90M) [application/zip]
Saving to: ‘resumes.zip’

resumes.zip         100%[===================>]  89.95M  9.68MB/s    in 12s     

2023-11-19 03:55:27 (7.55 MB/s) - ‘resumes.zip’ saved [94318691/94318691]



In [ ]:
!unzip resumes.zip -d resumes/

Archive:  resumes.zip
replace resumes/899331070003075c580039ed1f6b4b45596b35.html? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


# Парсинг и предобработка

In [ ]:
from bs4 import BeautifulSoup
contents =''
with open('/content/resumes/00abf8fa0003f155f70039ed1f306b3461684d.html', 'r') as f:
        contents = f.read()
soup = BeautifulSoup(contents, "html.parser")

In [ ]:
def prepareString(sntr):
  i=sntr.split()
  for j in range(len(i)):
    if '-' in i[j]:
      uwu = i[j].split('-')
      uwu = uwu[0]
      i[j] = uwu
    if i[j].isnumeric():
        i[j] = ''
    if len(i[j]) <= 3:
      i[j] = ''
    if 'цех' in i[j]:
      i[j] = ''
    if 'разряд' in i[j]:
      i[j] = ''
    if 'кафедр' in i[j]:
      i[j] = ''
    if 'отряд' in i[j]:
      i[j] = ''
    if 'категор'in i[j]:
      i[j] = ''
    if 'класс' in i[j]:
      i[j] = ''
    if 'отдел' in i[j]:
      i[j] = ''
    i[j]=i[j].strip()
  return ''.join(e+' ' for e in i)
def prepareProfessions(strng):
    arrayChanges =[

        ("зам.", "заместитель "),
        ("зам ", "заместитель "),
        ("нач.", "начальник "),
        ("нач ", "начальник "),
        ("рук.", "руководитель "),
        ("ген. ", "генеральный "),
        ("гендир ", "генеральный директор"),
        ("главбух", "главный бухгалтер"),
        ("глав.", "главный "),
        ("ветврач", "ветеринарный врач"),
        ("вет.", "ветеринарный "),
        ("–", "-"),
        (" - ", "-"),
        (" -", "-"),
        ("- ", "-"),
        ("  ", " "),
        ("  ", " "),
        ("машинист фронтального", "водитель фронтального"),

    ]
    strng=strng.lower()
    for f, c in arrayChanges:
        strng=strng.replace(f, c)
    result=strng
    if (result.find("(")>0 and result.find(")")>0):
        temp = ""
        not_closed = False
        for i in range(len(result)):
            if result[i] == "(":
                not_closed = True
            elif result[i] == ")":
                not_closed = False
            else:
                if not not_closed:
                    temp += result[i]
        result = temp

    if (result.find("«")>0 and result.find("»")>0):
        temp = ""
        not_closed = False
        for i in range(len(result)):
            if result[i] == "«":
                not_closed = True
            elif result[i] == "»":
                not_closed = False
            else:
                if not not_closed:
                    temp += result[i]
        result = temp
    if (result.find(",") > 0) :
        result=result.partition(',')[0]
    if (result.find(";") > 0) :
        result=result.partition(';')[0]
    if (result.find(".") > 0) :
        result=result.partition('.')[0]
    if (result.find("/") > 0) :
        result=result.partition('/')[0]
    if (result.find(" по ") > 0) :
        result=result.partition(' по ')[0]
    if (result.find("начальник ") > -1 or result.find("руководитель ") > -1):
        if (result.find("отдела") > -1):
            result = "начальник отдела"
        elif (result.find("службы") > -1):
            result = "начальник службы"
        elif (result.find("управления") > -1):
            result = "начальник управления"
        elif (result.find("подразделения") > -1):
            result = "начальник подразделения"
        elif (result.find("группы") > -1):
            result = "начальник группы"
    return prepareString(result.strip()).strip()

In [ ]:
import os
import re
from tqdm.auto import tqdm
path='/content/resumes/'
all_files=os.listdir(path)
onlyHTML=[x for x in all_files if x.find('html')]
allNames=[]
for i in tqdm(onlyHTML):
    contents =''
    with open(f'/content/resumes/{i}', 'r') as f:
        contents = f.read()
    soup = BeautifulSoup(contents, "html.parser")
    lst_def=[el.text.lower() for el in soup.findAll('div', attrs={'data-qa': 'resume-block-experience-position'})]
    lst = [prepareProfessions(el.text) for el in soup.findAll('div', attrs={'data-qa': 'resume-block-experience-position'})]
    #st = ''.join(e for e in lst_def)
    #if len((''.join((e.strip()) for e in lst_def)).split())==1:
        #lst = [''.join(e+' ' for e in lst_def)]
    allNames.append({'professions': lst, 'professions_class': lst_def})

  0%|          | 0/1001 [00:00<?, ?it/s]

# Анализ при помощи ruBert-tiny

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
model = AutoModel.from_pretrained("cointegrated/rubert-tiny2")
# model.cuda()  # uncomment it if you have a GPU

def embed_bert_cls(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()

tokenizer_config.json:   0%|          | 0.00/401 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.74M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/118M [00:00<?, ?B/s]

(312,)


In [ ]:
import numpy as np
arr = []
for i in tqdm(allNames):
    ndic=dict()
    for j in range(len(i['professions'])):
        t = embed_bert_cls(i['professions'][j], model, tokenizer)
        ndic[i['professions'][j]]=t
    arr.append({"prof": i['professions'][j], "vec": ndic})


  0%|          | 0/1001 [00:00<?, ?it/s]

In [90]:
from numpy.linalg import norm

stopwords=['мебель', 'яичница', 'грязь', 'диван', 'наушники']
stoparr = [embed_bert_cls(e, model, tokenizer) for e in stopwords]

# compute cosine similarity
profdict=dict()
for i in tqdm(arr):
    for k in i['vec'].keys():
        v=i['vec'][k]
        profdict[k]=v
        for stp in stoparr:
            cosine = np.dot(v,stp)/(norm(stp)*norm(v))
            profdict[k]=profdict[k]-cosine*stp

#profSimDict = dict()
#for k1 in profdict.keys():
    #v1 = profdict[k1]
    #ndct=dict()
    #for k2 in profdict.keys():
        #v2 = profdict[k2]
        #ndct[k2]=np.dot(v1,v2)/(norm(v1)*norm(v2))
    #profSimDict=ndct
# //


  0%|          | 0/1001 [00:00<?, ?it/s]

# Кластеризация

In [97]:
import pandas as pd
df = pd.DataFrame(profdict).transpose()

In [152]:
from sklearn.manifold import TSNE

from sklearn.datasets import load_iris
from numpy import reshape
import seaborn as sns
import pandas as pd
ts=TSNE()
projections = ts.fit_transform(df)


In [132]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
K=10
kmeans = KMeans(K)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
dfKm=df
dfKm['X']=np.transpose(projections, (1, 0))[0]
dfKm['Y']=np.transpose(projections, (1, 0))[1]
dfKm.columns = dfKm.columns.astype(str)
dfKmScaled= pd.DataFrame(scaler.fit_transform(dfKm), columns=dfKm.columns)

kmeans.fit(dfKmScaled)
import plotly.express as px
fig = px.scatter(
    projections, x=0, y=1,
    color=kmeans.labels_
)
fig.show()

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



In [123]:
dfKm

,0,1,2,3,4,5,6,7,8,9,...,304,305,306,307,308,309,310,311,X,Y
главный ветеринарный врач,0.037019,0.066666,0.068877,-0.067212,0.016641,-0.042515,-0.038029,0.003751,0.021279,-0.061579,...,-0.048006,0.063852,-0.051532,-0.032878,0.018561,-0.035401,-0.065574,0.147301,56.587273,1.487073
совместительству: ветеринарный врач огвк,0.085877,0.053591,0.030418,-0.070801,0.080655,-0.074926,-0.031468,-0.010515,0.029527,-0.066944,...,0.025376,0.082110,-0.035409,-0.012387,0.008432,-0.016103,-0.005487,0.156248,61.519455,6.295817
ветеринарный врач,0.057104,0.056604,0.077949,-0.019580,0.013246,-0.065840,-0.052664,-0.004519,0.042710,-0.066621,...,-0.007738,0.074211,-0.056752,-0.027562,-0.006043,-0.028710,-0.062737,0.151384,55.259773,10.222131
заведующий ветеринарным участком,0.082612,0.023045,0.065432,-0.003373,0.007225,-0.027470,-0.017413,-0.017834,0.045176,-0.015482,...,-0.008576,0.093250,-0.074511,-0.058648,0.036102,-0.013097,-0.043066,0.176285,49.378384,3.558079
ветеринарный врач разделки индейки,0.031488,0.023949,0.095424,-0.026810,0.049215,-0.042278,-0.051152,0.026442,0.001148,-0.046612,...,0.021036,0.067253,-0.036706,-0.030466,0.003678,0.002013,-0.074876,0.084396,60.236561,4.652287
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
временно исполняющий обязанности главного механика,-0.026757,0.036030,0.011654,-0.034096,0.088821,-0.051441,-0.066400,-0.058645,0.117954,0.012006,...,0.029826,0.068916,-0.077922,-0.049577,0.015067,-0.009986,-0.022578,0.061402,24.237169,-3.735276
начальник комбикормового,0.090821,0.039222,0.015336,0.050797,0.076128,-0.061170,-0.049236,-0.041252,0.001136,-0.022331,...,-0.005595,0.141822,-0.076610,-0.041100,-0.086634,-0.033869,-0.069109,0.160520,-10.945481,6.720959
индивидуальная трудовая деятельность,0.065827,0.049100,0.040465,0.064019,0.025152,-0.065521,-0.023040,-0.039046,0.045706,-0.045297,...,-0.003198,0.126057,-0.030191,-0.041307,-0.003418,-0.092765,0.019972,0.064708,9.107048,6.615596
телефонист,0.067940,0.106366,0.042747,0.100101,-0.000739,-0.029831,-0.058899,-0.101822,0.093700,-0.027735,...,0.063144,0.164623,-0.089829,-0.030719,-0.019044,-0.023076,-0.084863,0.162725,-46.777760,-5.066834


# Сериализация в JSON для отображения

In [143]:
jsnGraphNodes=dict()
nodelist=[]
i = 0
nodedict=dict()
for k in profdict.keys():
    nodelist.append({ "data": { "id": f"node{i}", "name": f"Color{kmeans.labels_[i]}", "label": k } })
    nodedict[k]=i
    i+=1

In [146]:
j=0
edgelist=[]
for i in range(len(allNames)):
    e=allNames[i]
    for vali in range(1, len(e['professions'])):
        val = e['professions'][vali]
        valpr = e['professions'][vali-1]
        edgelist.append({ "data": { "id": f"edge{j}", "source": f"node{nodedict[val]}", "target": f"node{nodedict[valpr]}" } })
        j+=1

In [142]:
allNames[0]['professions']

{'professions': ['главный ветеринарный врач',
  'совместительству: ветеринарный врач огвк',
  'ветеринарный врач',
  'заведующий ветеринарным участком',
  'ветеринарный врач   разделки индейки',
  'ветеринарный врач'],
 'professions_class': ['главный ветеринарный врач',
  'по совместительству: ветеринарный врач огвк. фгис меркурий',
  'ветеринарный врач',
  'заведующий ветеринарным участком из 5 сёл. ветеринарный врач.',
  'ветеринарный врач в цехе разделки индейки',
  'ветеринарный врач']}

In [150]:
import json

output = {'nodes': nodelist, 'edges': edgelist}
outputJSON= json.dumps(output, ensure_ascii=False)

In [151]:

import json
with open('data.json', 'w', encoding='utf-8') as f:
    json.dump(output, f, ensure_ascii=False, indent=4)